## 라이브러리 로드

In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져옵니다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱합니다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러옵니다.
import pandas as pd

## 서울시 코로나19 발생동향
* [코로나19](http://www.seoul.go.kr/coronaV/coronaStatus.do)

## requests 로 html 문서 받아오기

In [2]:
# 크롤링 할 사이트
base_url = "http://www.seoul.go.kr/coronaV/coronaStatus.do?menu_code=01"
response = requests.get( base_url )
response

<Response [200]>

In [3]:
if response.status_code == 200:
    soup = bs(response.text, 'html.parser')

str(soup)[:1000]

'\n<!DOCTYPE HTML>\n\n<html lang="ko">\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport">\n<title>서울시 코로나19</title>\n<meta content="서울시 코로나19 통합정보 사이트로 안전·방역(발생동향, 클린존, 선별진료소, 일일브리핑, 공정 공급마스크&amp;홍보물, 보도자료, 소식지, 신고(응답소)), 생활정보(주요뉴스, 생활경제 지원, 온라인 문화생활, 팩트체크, 심리지원단, 정책 제안하기), 시민 참여(잠시멈춤 캠페인, 온서울캠페인, 시민제안)등의 정보를 제공하고 있습니다." name="description"/>\n<meta content="website" property="og:type"/>\n<meta content="서울시 코로나19 통합사이트" property="og:title"/>\n<meta content="서울시 코로나19 통합정보 사이트로 안전·방역(발생동향, 클린존, 선별진료소, 일일브리핑, 공정 공급마스크&amp;홍보물, 보도자료, 소식지, 신고(응답소)), 생활정보(주요뉴스, 생활경제 지원, 온라인 문화생활, 팩트체크, 심리지원단, 정책 제안하기), 시민 참여(잠시멈춤 캠페인, 온서울캠페인, 시민제안)등의 정보를 제공하고 있습니다." property="og:description"/>\n<meta content="https://www.seoul.go.kr/coronaV/coronaStatus.do" property="og:url"/>\n<meta content="http://www.seoul.go.kr/res_newseoul/images/corona/pic_facebook_20200325.jpg" property="og:image"/>\n<meta con

## 서울 확진자 현황

In [4]:
#move-cont1 > div:nth-child(2) > div

#move-cont1 > div:nth-child(2) > div
cont_page = soup.select("#move-cont1 > div > div.cont-page-wrap > div.cont-page")
len(cont_page)

4

In [5]:
trs_ex =  cont_page[0].select("tr")
trs_ex[:3]

[<tr>
 <th scope="col">연번</th>
 <th scope="col">환자</th>
 <th scope="col">확진일</th>
 <th scope="col">성별(출생년)</th>
 <th scope="col">거주지</th>
 <th scope="col">여행력</th>
 <th scope="col">접촉력</th>
 <th scope="col">조치사항</th>
 </tr>, <tr>
 <th scope="row">361</th>
 <td data-tit="환자 번호" scope="col">확인중</td>
 <td data-tit="확진일">3.25.</td>
 <td data-tit="성별(출생년)">여('02)</td>
 <td data-tit="거주지">강남구</td>
 <td data-tit="여행력">미국</td>
 <td data-tit="접촉력">해외접촉 추정</td>
 <td data-tit="조치사항">서북병원</td>
 </tr>, <tr>
 <th scope="row">360</th>
 <td data-tit="환자 번호" scope="col">9228</td>
 <td data-tit="확진일">3.25.</td>
 <td data-tit="성별(출생년)">여('02)</td>
 <td data-tit="거주지">강남구</td>
 <td data-tit="여행력">미국</td>
 <td data-tit="접촉력">해외접촉 추정</td>
 <td data-tit="조치사항">서북병원</td>
 </tr>]

## 행 데이터 리스트로 만들기

In [6]:
def parse_tr(trs):
    table = []
    for tr in trs:
        tds = tr.select("td")
        if len(tds) > 0 :
            row = []
            # 연번
            number = tr.select("th")[0].get_text()
            row.append(number)
            # 연번 외 데이터
            for td in tds:
                val = td.get_text()
                row.append(val)
            table.append(row)
    return table

parse_tr(trs_ex[:3])

[['361', '확인중', '3.25.', "여('02)", '강남구', '미국', '해외접촉 추정', '서북병원'],
 ['360', '9228', '3.25.', "여('02)", '강남구', '미국', '해외접촉 추정', '서북병원']]

## 페이지마다 가져오기

In [7]:
#move-cont1 > div:nth-child(2) > table
#move-cont1 > div:nth-child(2) > table.tstyle05.tstyleP > thead > tr > th:nth-child(1)
#cont-page2
table_result = []
for page in cont_page:
    trs = page.select("tr")
    result = parse_tr(trs)
    table_result.extend(result)

table_result[:5]

[['361', '확인중', '3.25.', "여('02)", '강남구', '미국', '해외접촉 추정', '서북병원'],
 ['360', '9228', '3.25.', "여('02)", '강남구', '미국', '해외접촉 추정', '서북병원'],
 ['359', '9171', '3.25.', "남('99)", '강남구', '미국', '해외접촉 추정', '서남병원'],
 ['358', '9204', '3.25.', "남('93)", '대전', '영국', '해외접촉 추정', '생활치료센터'],
 ['357', '9219', '3.25.', "여('00)", '강남구', '미국', '해외접촉 추정', '보라매병원']]

## 컬럼명 만들기

In [8]:
# 컬럼명 만들기
cols = trs_ex[0].select("th")
cols

[<th scope="col">연번</th>,
 <th scope="col">환자</th>,
 <th scope="col">확진일</th>,
 <th scope="col">성별(출생년)</th>,
 <th scope="col">거주지</th>,
 <th scope="col">여행력</th>,
 <th scope="col">접촉력</th>,
 <th scope="col">조치사항</th>]

In [9]:
col_name = []
for col in cols:
    col_name.append(col.get_text())
col_name

['연번', '환자', '확진일', '성별(출생년)', '거주지', '여행력', '접촉력', '조치사항']

## 데이터프레임으로 만들기

In [10]:
df = pd.DataFrame(table_result, columns=col_name)
print(df.shape)
df.head()

(361, 8)


,연번,환자,확진일,성별(출생년),거주지,여행력,접촉력,조치사항
0,361,확인중,3.25.,여('02),강남구,미국,해외접촉 추정,서북병원
1,360,9228,3.25.,여('02),강남구,미국,해외접촉 추정,서북병원
2,359,9171,3.25.,남('99),강남구,미국,해외접촉 추정,서남병원
3,358,9204,3.25.,남('93),대전,영국,해외접촉 추정,생활치료센터
4,357,9219,3.25.,여('00),강남구,미국,해외접촉 추정,보라매병원


## CSV 파일로 저장하기

In [11]:
df.to_csv("covid-19-seoul.csv", index=False)

## CSV 파일 읽어오기

In [12]:
pd.read_csv("covid-19-seoul.csv").head(10)

,연번,환자,확진일,성별(출생년),거주지,여행력,접촉력,조치사항
0,361,확인중,3.25.,여('02),강남구,미국,해외접촉 추정,서북병원
1,360,9228,3.25.,여('02),강남구,미국,해외접촉 추정,서북병원
2,359,9171,3.25.,남('99),강남구,미국,해외접촉 추정,서남병원
3,358,9204,3.25.,남('93),대전,영국,해외접촉 추정,생활치료센터
4,357,9219,3.25.,여('00),강남구,미국,해외접촉 추정,보라매병원
5,356,9170,3.25.,여('45),강서구,-,콜센터직원 접촉,서남병원
6,355,9175,3.25.,여('99),서초구,프랑스,해외접촉 추정,보라매병원
7,354,9194,3.25.,여('99),서초구,미국,해외접촉 추정,보라매병원
8,353,9187,3.25.,남('80),금천구,-,확인중,보라매병원
9,352,9183,3.25.,여('00),서초구,미국,해외접촉 추정,보라매병원


In [13]:
df["접촉력"].value_counts()

구로구 콜센터 관련    60
해외접촉 추정       54
확인중           33
콜센터직원 접촉      33
타시도 확진자 접촉    22
              ..
#6937 접촉자      1
#6 접촉(추정)      1
#3 접촉          1
#6921접촉자       1
구로구 콜센터 접촉     1
Name: 접촉력, Length: 72, dtype: int64

In [14]:
df["조치사항"].value_counts()

서울의료원          96
서남병원           72
보라매병원          41
서울의료원(퇴원)      21
생활치료센터         15
국립중앙의료원(퇴원)    13
서북병원           12
국립중앙의료원        11
서남병원(퇴원)       11
서울대학교병원(퇴원)    10
보라매병원(퇴원)       8
서울성모병원          4
중앙대학교병원(퇴원)     4
서북병원(퇴원)        3
중앙대학교병원         3
삼육서울병원          3
순천향서울병원         3
타시도이관           3
타시도 이관          3
타시도이관(퇴원)       3
상계백병원(퇴원)       3
한일병원(퇴원)        2
확인중             2
한양대학교병원(퇴원)     2
한일병원            2
서울백병원           1
서울대학교병원         1
서울아산병원          1
세브란스병원          1
상계백병원           1
고대구로병원(퇴원)      1
고대안암병원          1
상계병원            1
고대구로병원          1
강남세브란스병원        1
국립중앙의료병원        1
Name: 조치사항, dtype: int64